# Sheet 7

Johannes van de Locht, Finn Kalvelage, Anna Beckers

## Task 1

## Task 2

### a) What is the difference between geometric zoom and detail zoom in the system?

Geometric zoom maps a part of the adjacency matrix from the current detail level to the viewport. This can be navigated by the user through zooming in and out (changing the amount of cells displayed in the viewport) or through horizontal and vertical translation. This makes the geometric zoom continious.

The detail zoom switches through the different detail levels and is therefore discrete.

### b) The paper suggests eight different ways to visualize aggregated edge information. Which of them are actually used in the screenshots in Figures 1 and 9? 

The images in figure 1 both use *step histograms* for cells with edges while the left image of figure 9 uses the *average* visualization and the right image looks like it uses *color shading* though the used color shows no differences between cells.

### c) What is the role of tile management within the ZAME system? What is an LRU cache? 

As the data structure (especially for bigger datasets) can be to large for the RAM or VRAM (atleast at the time of the release of the paper) not the complete adjacency matrix let alone each detail level could be present in the cache (RAM or VRAM). The tile manager is used for the loading the correct detail level and has the ability to temporarily present a lower detail level during the loading process. The policy for choosing which tiles to keep in the caches is the least recently used (LRU) policy which discards the tile that wasn't used for the longest when a new tile needs to be cached and the cache is already full.

### d) How does ZAME aggregate nominal attributes? Why is that problematic?

As there is no immediatly obvious way for nominal attributes to be aggregated ZAME uses the first label that represents the whole aggregate. This method fails to capture the diversity of the whole aggregate and can also be quite arbitrary depending on the ordering of labels.

### e) Why is the Traveling Salesman Problem relevant to adjacency matrix based graph visualization?

ZAME uses an aproximation algorithm for the TSP to find good orderings of adjacency matrices for graphs with relativly dissimilar weighted edges. The desired ordering of vertices to build good local structures probably arises from the need to travel to each vertex of a highly interconnected cluster in quick succession to achieve the shortest route.

### f) In the pseudocode listed in the paper’s Figure 4, some modifications are highlighted in boldface, on lines starting with a bar. What is the purpose of these modifications?

The bold-faced parts of the algorithm exist to spread the pivot point equally around the subtrees. Without the penalization the algorithm would prioritize the 2 largest subtrees and therefore misrepresent the rest of the data.